# 编译运行 GEM5

In [ ]:
%%bash

pushd ../ && source env.sh && popd

check_env() {
    if [[ -z "${GEM5_HOME}" ]]; then
        echo "Error: No GEM5_HOME found" 1>&2
        return 1
    fi
}

prepare_gem5() {
    if [ -f "$GEM5_HOME/ext/dramsim3/DRAMsim3/libdramsim3.so" ]; then
        echo "DRAMSim3 already built, skip building DRAMSim3"
        return 0
    fi
    pushd $GEM5_HOME && \
    cd ext/dramsim3 && \
    (stat DRAMsim3 || git clone https://github.com/umd-memsys/DRAMSim3.git DRAMsim3) && \
    cd DRAMsim3 && mkdir -p build && cd build && cmake .. && make -j `nproc` && \
    popd
}

build_gem5() {
    if [ -f "$GEM5_HOME/build/RISCV/gem5.opt" ]; then
        echo "gem5 already built, skip building gem5"
        return 0
    fi
    pushd $GEM5_HOME && \
    scons build/RISCV/gem5.opt --linker=mold -j `nproc` && \
    popd
}

prepare_data_proc() {
    # Validated commit for tutorial: 4000c092b8bde21fd4aa493f9907fa100dbcb3fc
    pushd xs-gem5 && \
    (stat gem5_data_proc || git clone https://github.com/shinezyy/gem5_data_proc.git) && \
    pushd gem5_data_proc && \
    git reset --hard 4000c092b8bde21fd4aa493f9907fa100dbcb3fc && \
    pip3 install -r requirements.txt && \
    popd && \
    popd
}

check_env && prepare_gem5 && build_gem5 && build_nemu_diff && prepare_data_proc